In [1]:
import requests
import csv
import json
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
from plotly.subplots import make_subplots

In [8]:
# site_id = '06730200' #boulder creek near boulder
site_id = '06719505' #clear creak near golden
# site_id = '06710385' #bear creek abov evergreen
# site_id = '06710605' #bear creek above bear creek lake
# site_id = '06741510' # BIG THOMPSON RV AT loveland
# site_id = '06752260' #CACHE LA POUDRE RIVER AT FORT COLLINS
# site_id = '402114105350101' #BIG THOMPSON BL MORAINE PARK NR ESTES PARK

weeks = 1040
years = int(weeks / 52)

hist_req = requests.get("https://waterservices.usgs.gov/nwis/dv/?format=json&sites={}&period=P{}W&statCd=00003&siteStatus=all".format(site_id, weeks))
recent_req = requests.get("https://waterservices.usgs.gov/nwis/dv/?format=json&sites={}&startDT=2020-01-01&statCd=00003&siteStatus=all".format(site_id))

# recent data
res = recent_req.json()

rec_days = []
rec_cfss = []

site_name = res["value"]["timeSeries"][0]["sourceInfo"]["siteName"]
lat = res["value"]["timeSeries"][0]["sourceInfo"]["geoLocation"]["geogLocation"]["latitude"]
lon = res["value"]["timeSeries"][0]["sourceInfo"]["geoLocation"]["geogLocation"]["longitude"]

for vals in res["value"]["timeSeries"][0]["values"][0]["value"]:
    cfs = round(float(vals["value"]),0)
    day = vals["dateTime"]
    date = datetime.strptime(day.split("T")[0], "%Y-%m-%d")
    doy = datetime.strftime(date, "%B %d")
    montInt = datetime.strftime(date, "%m")
    rec_days.append("{} {}".format(montInt, doy))
    rec_cfss.append(cfs)

# historical data
res = hist_req.json()
his_days = []
his_cfss = []
for vals in res["value"]["timeSeries"][0]["values"][0]["value"]:
    cfs = round(float(vals["value"]),0)
    day = vals["dateTime"]
    date = datetime.strptime(day.split("T")[0], "%Y-%m-%d")
    doy = datetime.strftime(date, "%B %d")
    montInt = datetime.strftime(date, "%m")
    his_days.append("{} {}".format(montInt, doy))
    his_cfss.append(cfs)

In [9]:
rec_d = {'day': rec_days, 'cfs': rec_cfss}
rec_df = pd.DataFrame(data=rec_d)

his_d = {'day': his_days, 'cfs': his_cfss}
his_df = pd.DataFrame(data=his_d)

rec_cfs_df = rec_df.groupby(['day']).agg({"cfs": ["mean"]}).reset_index()
cols = rec_cfs_df.columns.tolist()
rec_cfs_df["mean_cfs"] = rec_cfs_df[cols[1]]

his_cfs_df = his_df.groupby(['day']).agg({"cfs": ["mean"]}).reset_index()
cols = his_cfs_df.columns.tolist()
his_cfs_df["mean_cfs"] = his_cfs_df[cols[1]]

max_cfs_df = his_df.groupby(['day']).agg({"cfs": ["max"]}).reset_index()
cols = max_cfs_df.columns.tolist()
max_cfs_df["max_cfs"] = max_cfs_df[cols[1]]

min_cfs_df = his_df.groupby(['day']).agg({"cfs": ["min"]}).reset_index()
cols = min_cfs_df.columns.tolist()
min_cfs_df["min_cfs"] = min_cfs_df[cols[1]]

rec_cfs_df[['month', 'new_day']] = rec_cfs_df.day.str.split(expand=True, n=1)
his_cfs_df[['month', 'new_day']] = his_cfs_df.day.str.split(expand=True, n=1)
max_cfs_df[['month', 'new_day']] = his_cfs_df.day.str.split(expand=True, n=1)
min_cfs_df[['month', 'new_day']] = his_cfs_df.day.str.split(expand=True, n=1)

In [10]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=his_cfs_df['new_day'], y=his_cfs_df['mean_cfs'], name='Mean CFS for the Day<br>(Last {} Years)'.format(years),
                         line=dict(color='orange', width=2)))

fig.add_trace(go.Scatter(x=max_cfs_df['new_day'], y=max_cfs_df['max_cfs'], name='Max CFS for the Day<br>(Last {} Years)'.format(years),
                         line=dict(color="rgb(231, 29, 53)", width=1, dash='dot')))

fig.add_trace(go.Scatter(x=min_cfs_df['new_day'], y=min_cfs_df['min_cfs'], name='Min CFS for the Day<br>(Last {} Years)'.format(years),
                         line=dict(color="rgb(30, 100, 46)", width=1, dash='dot')))

fig.add_trace(go.Scatter(x=rec_cfs_df['new_day'], y=rec_cfs_df['mean_cfs'], name='Mean CFS for the Day<br>(2020)',
                         line=dict(color="navy", width=2)))

fig.update_layout(
    title_text="CFS for {}<br>Coordinates: {}, {}".format(site_name.title(), lat, lon),
    font=dict(family='Courier New, monospace', size=12, color='#7f7f7f'),
    yaxis_title="Cubic Feet per Second (CFS)"
)
fig.show()

In [5]:
locs = pd.DataFrame({'lat': [lat], 'lon': [lon], 'site_name': [site_name]})

m_fig = go.Figure(go.Scattermapbox(
    lat=locs["lat"],
    lon=locs["lon"],
    mode="markers",
    marker=go.scattermapbox.Marker(
        size=20,
        color='red'
    ),
    text=locs["site_name"]
))

m_fig.update_layout(
    mapbox_style="open-street-map",
    mapbox=dict(
        center=dict(
        lat=lat,
        lon=lon
        ),
        zoom=13
    )
    
)
m_fig.show()

In [11]:
locs = pd.DataFrame({'lat': [lat], 'lon': [lon], 'site_name': [site_name]})

fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.25, 0.75],
    specs=[[{"type": "mapbox"}, {"type": "xy"}]]
)

fig.add_trace(go.Scattermapbox(
    lat=locs["lat"],
    lon=locs["lon"],
    mode="markers",
    showlegend=False,
    marker=go.scattermapbox.Marker(
        size=20,
        color='red'
    ),
    text=locs["site_name"]
    ),row=1,col=1).update_layout(
        mapbox_style="open-street-map",
        mapbox=dict(
            center=dict(
            lat=lat,
            lon=lon
            ),
            zoom=11
        )
    
)

fig.add_trace(go.Scatter(x=his_cfs_df['new_day'], y=his_cfs_df['mean_cfs'], name='Mean CFS for the Day<br>(Last {} Years)'.format(years),
                         line=dict(color='orange', width=2)),row=1,col=2)

fig.add_trace(go.Scatter(x=max_cfs_df['new_day'], y=max_cfs_df['max_cfs'], name='Max CFS for the Day<br>(Last {} Years)'.format(years),
                         line=dict(color="rgb(231, 29, 53)", width=1, dash='dot')),row=1,col=2)

fig.add_trace(go.Scatter(x=min_cfs_df['new_day'], y=min_cfs_df['min_cfs'], name='Min CFS for the Day<br>(Last {} Years)'.format(years),
                         line=dict(color="rgb(30, 100, 46)", width=1, dash='dot')),row=1,col=2)

fig.add_trace(go.Scatter(x=rec_cfs_df['new_day'], y=rec_cfs_df['mean_cfs'], name='Mean CFS for the Day<br>(2020)',
                         line=dict(color="navy", width=2)),row=1,col=2).update_layout(
                                                                            title_text="CFS for {}<br>Coordinates: {}, {}".format(site_name.title(), lat, lon),
                                                                            font=dict(family='Courier New, monospace', size=10, color='#7f7f7f'),
                                                                            yaxis_title="Cubic Feet per Second (CFS)")

fig.show()

In [12]:
fig.write_html(r"Z:\Dev\CFS_Charts\cfs_20.html")